 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/05/05.ipynb#Načítání-dat-ze-souboru" data-toc-modified-id="Načítání-dat-ze-souboru-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Načítání dat ze souboru</a></span></li></ul></div>

# Manipulace dat a popisná statistika

V předchozích dvou lekcích jsme si ukázali [základní principy programování v R](../03/) a [hlavní datové struktury](../04/). Nyní si ukážeme jak tyto dvě lekce propojit a využít tyto znalosti k jednoduché analýze různých datových souborů. Využijeme k tomu nejrůznějších principů popisné statistiky a ukážeme si též možnosti základní datové vizualizace pro jednoduchou exploraci dat.

## Načítání dat ze souboru

Data většinou nebudete do tabulek a matic vkládat manuálně ale z externích zdrojů, kterými mohou být internetové stránky, nejrůznější databáze nebo obyčejné textové soubory. V R existuje mnoho možností, kterými lze data snadno získat z mnoha zdrojů. My jsme si zatím ukázali jak jednoduše přistupovat k testovacím datovým množinám, které jsou v R přístupné přímo jako proměnné (např. `volcano` nebo `mtcars`). Nejčastěji ale budete jako uživatel chtít načíst do R vlastní data a to ve formě textových souborů. V této lekci si tedy ukážeme jak na to.

V adresáři s touto lekcí se nachází dva soubory `ufocase.info` a `ufocase.bat`. Soubor `ufocase.info` obsahuje popis jedné z [volně dostupných datových množin](http://www.stat.ufl.edu/~winner/datasets.html) na stránkách Larryho Winnera z University of Florida. Tento soubor vypadá následovně:

```
Dataset:  ufocase.dat

Source:  www.ufocasebook.com

Description:  UFO/Alien Cases, including categorical outcomes:
Presence/Absence of physical effects, multimedia (picture/video),
ET contact, and abduction.

Variable/Columns

Year 1-4 
Name of sighting/event  7-56
Location   60-72
Physical effects 75   /* 1=Yes, 0=No  */
Multimedia  79  /*  1=Yes, 0=No  */
ET contact   83  /* 1=Yes, 0=No  */
Abduction   87   /*  1=Yes, 0=No  */
```

Soubor `ufocase.bat` pak obsahuje samotná data a jeho prvních pět řádků vypadá následovně:

```
1865  The Great Airships                                   USA            1   0   1  0
1878  Denison, Texas Daylight UFO                          USA            0   0   0  0
1897  The Aurora, Texas Crash                              USA            1   0   1  0
1930  Alien Picture from Alaska                            Alaska         0   1   1  0
1937  Discovery of the Dropa Discs                         China          0   0   0  0
```

Z popisku a samotných dat tedy vidíme, že se jedná o záznamy pozorování neidentifikovatelných létajících objektů (UFOs) na různých místech. K dispozici jsou ke každému případu informace o roku (`Year`), kdy k událostem došlo, písemný popisek události `Name of sighting/event`, název lokace (`Location`), kde k pozorování došlo a informace o tom, zda během pozorování došlo k nějakým fyzickým projevům (`Physical effects`), zda jsou k dispozici multimediální nahrávky z události (`Multimedia`), zda došlo ke kontaktu (`ET contact`) a zda došlo přímo k unesení mimozemšťanem (`Abduction`).

Soubor nás svojí strukturou tedy přímo vybízí k tomu, abychom z něj vytvořili `data.frame` a nějakým způsobem data analyzovali. Má to ovšem háček. Pro nás je struktura dokumentu sice jasná, ale jak to říci našemu programu? Jednotlivé sloupce jsou totiž od sebe oddělená v podstatě libovolným počtem mezer. Situaci navíc ještě komplikuje druhý sloupec s popisem události, který má mezery mezi slovy. Pojďme si tedy napsat program, který tento soubor nejprve zanalyzuje řádku po řádce a pokusí se na základě jednoduchého pravidla rozlišit záznamy z jendotlivých sloupců.

Načtení souboru v R probíhá následovně. Nejprve je třeba vytvořit připojení k souboru, ze kterého budeme jednotlivé řádky číst:

In [1]:
connection  <- file("ufocase.dat", open = "r")

Tímto způsobem tedy řekneme, že chceme orevřít soubor z daného umístění a budeme jej pouze číst (`open = "r"`). Funkce `file` umožňuje krom souborů na disku otevírat soubory i z dalších umístění a podporuje i některé běžné kompresní formáty (viz nápověda, `help(file)`). Proměnnou jsme si nazvali `connection`, protože se po technické stránce jedná o otevření připojení k určitému proudu dat, který budeme postupně číst.

Soubor máme takto tedy připravený pro čtení. V našem případě chceme soubor číst řádku po řádce, k čemuž slouží v R funkce `readlines`, které dáme k dispozici naše připojení a počet řádek, který chceme naráz ze souboru načíst:

In [2]:
readLines(connection, n = 5)

[1] "1865  The Great Airships                                   USA            1   0   1   0"
[2] "1878  Denison, Texas Daylight UFO                          USA            0   0   0   0"
[3] "1897  The Aurora, Texas Crash                              USA            1   0   1   0"
[4] "1930  Alien Picture from Alaska                            Alaska         0   1   1   0"
[5] "1937  Discovery of the Dropa Discs                         China          0   0   0   0"

Toto je tedy prvních pět řádek souboru. Zde je dalších pět:

In [3]:
readLines(connection, n = 5)

[1] "1939  Alien Bodies Confirmed (Cordell Hull)                USA            0   0   1   0"
[2] "1941  Missouri Crash & Retrieval                           USA            1   1   1   0"
[3] "1942  The Battle of Los Angeles                            USA            1   1   0   0"
[4] "1944  World War II Foo-Fighters                            Europe         0   1   0   0"
[5] "1945  Lt. Gorman Chases a UFO                              USA            0   0   0   0"

Všimněte si, že funkce `readLines` vrací pokaždé další a další řádky. Je to tím, že objekt, který se skrývá pod proměnnou `connection`, si udržuje svůj vnitřní stav a při každém zavolání začne vracet data od té části souboru, kde dříve skončil. 

Když už připojení nepotřebujeme měli bychom ho uzavřít:

In [4]:
close(connection)

Tak zajistíme, že se připojení uzavře a není z něj již možné dál číst:

In [5]:
readLines(connection, n = 5)

ERROR: Error in readLines(connection, n = 5): invalid connection


Pokud chceme program znovu načíst, musíme tedy připojení znovu vytvořit:

In [6]:
connection <- file("ufocase.dat", open = "r")

Přistupme teď tedy již k samotné analýze souboru. Jelikož nevíme, kolik má soubor řádek musíme použít cyklus `while` nebo `repeat`. My zde použijeme cyklus `repeat`, který v tomto případě umožňuje trošku lépe čitelnější kód:

In [7]:
repeat {
    line <- readLines(connection, n = 1)
    if (length(line) == 0) break
    print(line)
}

close(connection)

[1] "1865  The Great Airships                                   USA            1   0   1   0"
[1] "1878  Denison, Texas Daylight UFO                          USA            0   0   0   0"
[1] "1897  The Aurora, Texas Crash                              USA            1   0   1   0"
[1] "1930  Alien Picture from Alaska                            Alaska         0   1   1   0"
[1] "1937  Discovery of the Dropa Discs                         China          0   0   0   0"
[1] "1939  Alien Bodies Confirmed (Cordell Hull)                USA            0   0   1   0"
[1] "1941  Missouri Crash & Retrieval                           USA            1   1   1   0"
[1] "1942  The Battle of Los Angeles                            USA            1   1   0   0"
[1] "1944  World War II Foo-Fighters                            Europe         0   1   0   0"
[1] "1945  Lt. Gorman Chases a UFO                              USA            0   0   0   0"
[1] "1947  The Kenneth Arnold Sighting                      